In [1]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`

import $ivy.$                                  

In [17]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.sql.{Row}
import org.apache.spark.sql.types._

import spark.implicits._

21/07/26 18:52:49 INFO BlockManagerInfo: Removed broadcast_2_piece0 on andersons-mbp:54253 in memory (size: 5.7 KiB, free: 4.6 GiB)
21/07/26 18:52:49 INFO BlockManagerInfo: Removed broadcast_3_piece0 on andersons-mbp:54253 in memory (size: 5.7 KiB, free: 4.6 GiB)


import org.apache.spark.sql.SparkSession

import org.apache.spark.sql.functions._

import org.apache.spark.sql.{Row}

import org.apache.spark.sql.types._


import spark.implicits._

In [10]:
val spark = SparkSession
    .builder()
    .appName("Spark SQL basic example")
    .master("local")
    .config("spark.executor.instances", "4")
    .config("spark.executor.memory", "2g")
    .getOrCreate()

21/07/26 18:48:33 WARN SparkSession$Builder: Using an existing SparkSession; some spark core configurations may not take effect.


spark: SparkSession = org.apache.spark.sql.SparkSession@301fffbb

In [11]:
val voteRDD = spark.sparkContext.parallelize(List(
    Row("Anderson", 20, true),
    Row("Fulano", 34, false),
    Row("Ciclano", 17, true),
    Row("Deltrano", 67, false)
))

val voteSchema = StructType(List(
    StructField("Name", StringType, true),
    StructField("Age", IntegerType, true),
    StructField("Voted", BooleanType, true)
))

voteRDD: org.apache.spark.rdd.RDD[Row] = ParallelCollectionRDD[4] at parallelize at cmd10.sc:1
voteSchema: StructType = StructType(
  StructField("Name", StringType, true, {}),
  StructField("Age", IntegerType, true, {}),
  StructField("Voted", BooleanType, true, {})
)

In [30]:
val vote_df = spark.createDataFrame(voteRDD, voteSchema)

21/07/26 18:58:54 INFO BlockManagerInfo: Removed broadcast_20_piece0 on andersons-mbp:54253 in memory (size: 15.1 KiB, free: 4.6 GiB)


vote_df: org.apache.spark.sql.package.DataFrame = [Name: string, Age: int ... 1 more field]

In [13]:
vote_df.show()

21/07/26 18:48:36 INFO SparkContext: Starting job: show at cmd12.sc:1
21/07/26 18:48:36 INFO DAGScheduler: Got job 1 (show at cmd12.sc:1) with 1 output partitions
21/07/26 18:48:36 INFO DAGScheduler: Final stage: ResultStage 1 (show at cmd12.sc:1)
21/07/26 18:48:36 INFO DAGScheduler: Parents of final stage: List()
21/07/26 18:48:36 INFO DAGScheduler: Missing parents: List()
21/07/26 18:48:36 INFO DAGScheduler: Submitting ResultStage 1 (MapPartitionsRDD[7] at show at cmd12.sc:1), which has no missing parents
21/07/26 18:48:36 INFO MemoryStore: Block broadcast_1 stored as values in memory (estimated size 12.8 KiB, free 4.6 GiB)
21/07/26 18:48:36 INFO MemoryStore: Block broadcast_1_piece0 stored as bytes in memory (estimated size 5.5 KiB, free 4.6 GiB)
21/07/26 18:48:36 INFO BlockManagerInfo: Added broadcast_1_piece0 in memory on andersons-mbp:54253 (size: 5.5 KiB, free: 4.6 GiB)
21/07/26 18:48:36 INFO SparkContext: Created broadcast 1 from broadcast at DAGScheduler.scala:1388
21/07/26 18

+--------+---+-----+
|    Name|Age|Voted|
+--------+---+-----+
|Anderson| 20| true|
|  Fulano| 34|false|
| Ciclano| 17| true|
|Deltrano| 67|false|
+--------+---+-----+



In [22]:
vote_df
    .filter($"Voted" && $"Name" =!= "Anderson")


21/07/26 18:55:18 INFO SparkContext: Starting job: show at cmd21.sc:1
21/07/26 18:55:18 INFO DAGScheduler: Got job 8 (show at cmd21.sc:1) with 1 output partitions
21/07/26 18:55:18 INFO DAGScheduler: Final stage: ResultStage 8 (show at cmd21.sc:1)
21/07/26 18:55:18 INFO DAGScheduler: Parents of final stage: List()
21/07/26 18:55:18 INFO DAGScheduler: Missing parents: List()
21/07/26 18:55:18 INFO DAGScheduler: Submitting ResultStage 8 (MapPartitionsRDD[21] at show at cmd21.sc:1), which has no missing parents
21/07/26 18:55:18 INFO MemoryStore: Block broadcast_8 stored as values in memory (estimated size 13.5 KiB, free 4.6 GiB)
21/07/26 18:55:18 INFO MemoryStore: Block broadcast_8_piece0 stored as bytes in memory (estimated size 5.8 KiB, free 4.6 GiB)
21/07/26 18:55:18 INFO BlockManagerInfo: Added broadcast_8_piece0 in memory on andersons-mbp:54253 (size: 5.8 KiB, free: 4.6 GiB)
21/07/26 18:55:18 INFO SparkContext: Created broadcast 8 from broadcast at DAGScheduler.scala:1388
21/07/26 1

+-------+---+-----+
|   Name|Age|Voted|
+-------+---+-----+
|Ciclano| 17| true|
+-------+---+-----+



In [29]:
vote_df
    .groupBy($"Voted")
    .agg(sum($"Age").alias("AgeAvg"))
    .explain()

== Physical Plan ==
*(2) HashAggregate(keys=[Voted#27], functions=[sum(cast(Age#26 as bigint))])
+- Exchange hashpartitioning(Voted#27, 200), ENSURE_REQUIREMENTS, [id=#263]
   +- *(1) HashAggregate(keys=[Voted#27], functions=[partial_sum(cast(Age#26 as bigint))])
      +- *(1) Project [Age#26, Voted#27]
         +- *(1) Scan ExistingRDD[Name#25,Age#26,Voted#27]


